In [1]:
import numpy as np
import torch.nn as nn
import torch
import scipy as sp
import math
import string
import re
import pickle
import transformers

/home/quarknova/miniconda3/envs/ml/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
source = "gilf/french-camembert-postag-model"

model = transformers.AutoModelForTokenClassification.from_pretrained(source)

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(source)

classifier = transformers.pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True)

In [4]:
sampleText = "Le persan (autonyme : fārsī, ou pārsī) est une langue indo-européenne. C'est la langue officielle et majoritaire de l'Iran (centre, centre-sud, nord-est), de l'Afghanistan et du Tadjikistan. Le persan est une langue notable mais minoritaire à Bahreïn, en Irak, à Oman, au Qatar et aux Émirats arabes unis. Le dari, dialecte du persan, également appelé persan afghan ou persan oriental, est une langue officielle en Afghanistan ; il est parlé également en Iran et au Pakistan. De même, le tadjik, autre dialecte du persan, est la langue officielle du Tadjikistan et également parlé au Kirghizistan, au Turkménistan, en Ouzbékistan, et dans une moindre mesure au Kazakhstan. Le persan ou l’une de ses langues-sœurs est également parlé en Azerbaïdjan, en Russie et en Ouzbékistan (minorité tadjike). Jadis, du XVIe au XIXe siècle, il fut la langue officielle de l'Empire moghol. Le persan fait partie du groupe indo-iranien de la famille des langues indo-européennes. C'est une langue du type « sujet-objet-verbe ». Les langues persanes s’écrivent surtout au moyen de l'alphabet arabo-persan, variante de l'alphabet arabe, bien qu'elles n'aient aucune parenté avec la langue arabe, dont elles diffèrent tant sur le plan de la grammaire que de la phonologie. Au Tadjikistan, en Russie, en Azerbaïdjan et en Ouzbékistan, le tadjik s’écrit au moyen de l'alphabet cyrillique."

In [5]:
%store -r embs

In [6]:
%store -r token_to_idx
%store -r idx_to_token

In [7]:
#Here, we are building the graph structure for TextRank

class Graph(object):
    def __init__(self, n, E, dir=False):
        self.n = n
        self.E = E
        self.score = [1 for i in range(n)]
        self.inV = [[] for i in range(n)]
        self.outV = [[] for i in range(n)]
        
        for e in E:
            w = 1 if len(e) == 2 else e[2]
            
            self.inV[e[1]].append([e[0], w])
            self.outV[e[0]].append([e[1], w])
            if not dir:
                self.inV[e[0]].append([e[1], w])
                self.outV[e[1]].append([e[0], w])
    
    #Part 9
    def textRank(self, d=0.85):
        maxShift = 1
        
        while(maxShift >= 1e-4):
            newScore = [(1-d) for i in range(self.n)]
            for i in range(self.n):
                for l, j in enumerate(self.inV[i]):
                    newScore[i] += d * self.score[j[0]] * self.inV[i][l][1]/sum([k[1] for k in self.outV[j[0]]])
            maxShift = max([abs(self.score[k] - newScore[k]) for k in range(self.n)])
            self.score = newScore
        return self.score

In [8]:
def tokenize(sentence, removeDup=False):
    rawData = classifier(sentence)
    cleanedData = [word for word in rawData if (word['entity_group'] in ['ADJ', 'ADJWH', 'ET', 'NC', 'NPP'])]
    tokens = [word['word'] for word in cleanedData]
    return (list(dict.fromkeys(tokens)) if removeDup else tokens)

In [9]:
#Variables

n = 32 #Dimensionality of the sentence vectors

In [2]:
#Here, we are coding the functions for the main computations

#Part 1
def sentenceSplit(t): #Takes a text, returns sentences
    sl = re.split("(?<=[.?!])", t) #Sentence splitting
    sl = [s if (s == "" or s[-1] not in [" ", "\n"]) else s[:-1] for s in sl]
    sl = [s if (s == "" or s[0] not in [" ", "\n"]) else s[1:] for s in sl]
    sl = [s for s in sl if len(tokenize(s)) != 0] #Cleaning up
    return sl

#Parts 2 and 3
#def sentenceParse(s): #Takes a sentence, returns a tree
    #return st
    
#Part 4
def vectorize(s): #Takes a sentence, returns a vector
    tokens = [t for t in tokenize(s) if t in idx_to_token]
    sv = np.array(embs[0])-np.array(embs[0])+np.sum(np.array([embs[token_to_idx[t]] for t in tokens]), 0)
    return np.squeeze(sv)

#Part 5
def topicVector(sv): #Takes sentence vectors, returns a topic vector
    return np.average(sv, 0)

#Part 6
def cleanVectors(sv): #Takes sentence vectors, returns cleaned sentence and topic vectors
    #Performing PCA
    svCentered = np.array([[i - np.average(s) for i in s] for s in sv])
    cov = np.cov(svCentered, rowvar=False)
    vals, vects = np.linalg.eig(cov)
    sortedEigs = sorted([(vals[i], vects[i]) for i in range(len(vals))], key = lambda x : x[0])
    principalComponent = sortedEigs[-1][1]
    #Cleaning the vectors
    svCleaned = np.array([s - (s.dot(principalComponent) * principalComponent) for s in sv])
    topic = topicVector(sv)
    topicCleaned = topic - (topic.dot(principalComponent) * principalComponent)
    return svCleaned, topicCleaned

#Part 7
def sentenceDist(i, j, sv, tv): #Takes sentence numbers and cleaned vectors, returns a distance
    cos_sim = nn.CosineSimilarity()
    return min(cos_sim(torch.Tensor([sv[i]]), torch.Tensor([sv[j]])), 2*cos_sim(torch.Tensor([tv]), torch.Tensor([sv[j]])))
    #args = sv[i].extend(sv[j]).extend(tv).append(sv[i].dot(sv[j]))
    #return distNet(args)

#Part 8 (+9)
def sentenceRank(t): #Takes a text, returns sentence rankings
    sl = sentenceSplit(t)
    sv = [vectorize(s) for s in sl]
    sv, tv = cleanVectors(sv)
    
    n = len(sl) #Graph construction
    gs = []
    for i in range(n):
        for j in range(i): #The graph is undirected, we avoid duplicates
            dist = sentenceDist(i, j, sv, tv)[0].item()
            gs.append([i, j, 1/max(dist, 1e-3)])
    g = Graph(n, gs, dir=False)
    return list(zip(sl, g.textRank()))

sentenceRank(sampleText)

NameError: name 'sampleText' is not defined

In [11]:
#Part 10
def summarize(t, length):
    unsortedList = [(k[0], k[1], i) for i, k in enumerate(sentenceRank(t))]
    sortedList = sorted(unsortedList, key=(lambda k: -k[1])) #Sorting
    sentences = [s for i, s in enumerate(sortedList) if (i == 0 or s[0] != sortedList[i-1][0])] #Removing duplicates
    
    summarySentences = []
    l = 0
    for s in sentences:
        l += len(tokenize(s[0]))
        if(l > length):
            break
        summarySentences.append(s)
    summary = [s[0] for s in sorted(summarySentences, key=(lambda k: k[2]))]
    return " ".join(summary)

print(sampleText)
print(summarize(sampleText, 50))

Le persan (autonyme : fārsī, ou pārsī) est une langue indo-européenne. C'est la langue officielle et majoritaire de l'Iran (centre, centre-sud, nord-est), de l'Afghanistan et du Tadjikistan. Le persan est une langue notable mais minoritaire à Bahreïn, en Irak, à Oman, au Qatar et aux Émirats arabes unis. Le dari, dialecte du persan, également appelé persan afghan ou persan oriental, est une langue officielle en Afghanistan ; il est parlé également en Iran et au Pakistan. De même, le tadjik, autre dialecte du persan, est la langue officielle du Tadjikistan et également parlé au Kirghizistan, au Turkménistan, en Ouzbékistan, et dans une moindre mesure au Kazakhstan. Le persan ou l’une de ses langues-sœurs est également parlé en Azerbaïdjan, en Russie et en Ouzbékistan (minorité tadjike). Jadis, du XVIe au XIXe siècle, il fut la langue officielle de l'Empire moghol. Le persan fait partie du groupe indo-iranien de la famille des langues indo-européennes. C'est une langue du type « sujet-ob

/home/quarknova/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:42: ComplexWarning: Casting complex values to real discards the imaginary part


De même, le tadjik, autre dialecte du persan, est la langue officielle du Tadjikistan et également parlé au Kirghizistan, au Turkménistan, en Ouzbékistan, et dans une moindre mesure au Kazakhstan. Jadis, du XVIe au XIXe siècle, il fut la langue officielle de l'Empire moghol. Le persan fait partie du groupe indo-iranien de la famille des langues indo-européennes. Au Tadjikistan, en Russie, en Azerbaïdjan et en Ouzbékistan, le tadjik s’écrit au moyen de l'alphabet cyrillique.
